In [43]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import mysql.connector
import re
import random

In [117]:
column_names = ['title','date']
new_df = pd.DataFrame(columns = column_names)

In [118]:
idx = 0

In [149]:
to_crawl = pd.read_csv("15일이상결측치있는달_2.csv")

In [150]:
target = to_crawl.sort_values(by = ['year','month'])
target = target.reset_index()
target.drop('index', axis = 1, inplace = True)
target

,year,month,missing_days
0,1989,6,21
1,1989,9,15
2,1989,11,20
3,1990,6,15
4,1990,10,24
5,1990,12,23
6,1991,3,20
7,1991,6,17
8,1991,8,23
9,1991,10,23


In [151]:
target.loc[0].month

6

In [218]:
count=48

In [219]:
chromedriver_path = 'C:/Users/kkksk/.wdm/drivers/chromedriver/win64/122.0.6261.129/chromedriver.exe'
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

In [ ]:
#cur_month = 1
#cur_year = 1995
#target_year = 2009

max_count = len(target)

#while cur_year <= target_year:
while count <= max_count:
    
    cur_year = target.loc[count].year
    cur_month = target.loc[count].month
    
    url = f"https://www.google.com/search?q=crude+oil&tbs=bkt:s,cdr:1,cd_min:{cur_month}/1/{cur_year},cd_max:{cur_month}/31/{cur_year}&tbm=bks"
    driver.get(url)
    
    time.sleep(10)
    
    next_button_time = 1
    
    while True:
        news_results = driver.find_elements(By.CSS_SELECTOR, 'div#rso>div')
        if not news_results: 
            break  
            
        for news_div in news_results:
            try:
                #Link = news_div.find_element(By.TAG_NAME, 'a').get_attribute('href')
                title_element = news_div.find_element(By.CSS_SELECTOR, 'a>h3')
                Title = title_element.text.strip(".")
                Title = re.sub(r"- \d+페이지", "", Title)
                news_elements = news_div.find_elements(By.CSS_SELECTOR, 'div')
                Date = ''  # Default Date value if not found
                if len(news_elements) >= 10:
                    info = news_elements[10].text  # Assuming 'info' is in the 11th element
                    info_text = info.split("·")
                    if len(info_text) > 1:  # Check if Date is available
                        Date = info_text[1].strip()
                
                #new_df.loc[idx] = [Title, Link, Date]
                new_df.loc[idx] = [Title, Date]
                idx += 1

            except Exception as e:
                print(e)
            
        try:
            if next_button_time < 10:
                next_button = driver.find_element(By.ID, 'pnnext')
                next_button.click()
                next_button_time +=1
                min_sleep_time = 1
                max_sleep_time = 10
                random_sleep_time = random.uniform(min_sleep_time, max_sleep_time)
                time.sleep(random_sleep_time)
            else :
                break
        except NoSuchElementException:
            break

    #if cur_month == 7:  
    #    cur_month = 1
    #    cur_year += 1
    #else: 
    #    cur_month += 6
    count+=1

In [214]:
count

56

In [215]:
new_df

,title,date
0,The Daily Gazette,1991-03-05 00:00:00
1,"Oil Companies Profits Are Seen As Immodest, If...",1991-03-06 00:00:00
2,Oil Companies Profit From War. Dallas,1991-03-06 00:00:00
3,Oil Prices Gush On News Soviets Will Slash Exp...,1991-03-19 00:00:00
4,Futures,1991-03-17 00:00:00
...,...,...
5905,Pipeline Crack Forces Shutdown.,1999. 11. 8.
5906,"Jump In Gas Prices, Airfares Likely To Follow ...",1999. 11. 19.
5907,Oil Prices Continue To Increase.,1999. 11. 20.
5908,Shippers Say Cargoes Should Be Safe. Millions ...,1999. 11. 26.


In [216]:
new_df['date'] = pd.to_datetime(new_df['date'])

In [217]:
new_df.iloc[-70]

title            Daily News 
date     1999-11-05 00:00:00
Name: 5840, dtype: object

In [185]:
new_df.to_csv("결측치수집데이터2.csv", index = False)